In [168]:
import pandas as pd
import numpy as np
from numpy import nan
import math

import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


df = pd.read_table('botswana.tsv')

df['nevermarr'] = df['agefm'].apply(lambda x : 1 if math.isnan(x) else 0)
del df['evermarr']
df.agefm.fillna(value=0, inplace=True)

#df.loc[df['nevermarr'] == 1, 'heduc'].fillna(value=-1, inplace=True)

In [169]:
df.loc[df['nevermarr'] == 1, 'heduc'] = -1

In [170]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [171]:
df.heduc.isnull().sum()

123

In [172]:
df['idlnchld_noans'] = df['idlnchld'].apply(lambda x : 0 if math.isnan(x) else 1)
df['heduc_noans'] = df['heduc'].apply(lambda x : 0 if math.isnan(x) else 1)
df['usemeth_noans'] = df['usemeth'].apply(lambda x : 0 if math.isnan(x) else 1)

df.idlnchld.fillna(value=-1, inplace=True)
df.heduc.fillna(value=-2, inplace=True)
df.usemeth.fillna(value=-1, inplace=True)

df = df[df['knowmeth'].notnull()]
df = df[df['electric'].notnull()]
df = df[df['tv'].notnull()]
df = df[df['radio'].notnull()]
df = df[df['bicycle'].notnull()]

In [173]:
df.shape

(4348, 18)

In [174]:
df.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
count,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000
mean,2.437443,27.393514,5.863155,4.466191,0.963431,0.553818,9.850736,1.730451,0.517249,0.140064,0.701702,0.092916,0.275299,0.523919,0.972861,0.971941,0.984821
std,2.401785,8.675630,3.922694,2.372879,0.187722,0.526808,10.897246,4.459982,0.499760,0.347094,0.457564,0.290348,0.446716,0.499485,0.162507,0.165160,0.122280
min,0.000000,15.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,19.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [175]:
4348*18

78264

In [176]:
str_regression = 'ceb ~ '
first = True
for column in df.columns:
    if column != 'ceb':
        plus = ' + '
        if first:
            plus = ''
        str_regression = str_regression + plus + column
        first = False

print str_regression

ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans


In [177]:
m1 = smf.ols(str_regression, data=df)
fitted1 = m1.fit()

In [178]:
fitted1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 10 Aug 2017   Prob (F-statistic):               0.00
Time:                        18:58:09   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.4796      0.341     -1.407      0.159      -1.148       0.189
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans            -0.6539      0.153     -4.286      0.000      -0.953      -0.355
heduc_noans                0.8724      0.145      6.026      0.000       0.589       1.156
usemeth_noans             -0.7652      0.196     -3.910      0.000      -1.149      -0.382
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         551.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [179]:
str_regression = 'ceb ~ '
first = True
for column in df.columns:
    if column in ['religion', 'radio', 'tv']:
        continue
    if column != 'ceb':
        plus = ' + '
        if first:
            plus = ''
        str_regression = str_regression + plus + column
        first = False
print str_regression
m2 = smf.ols(str_regression, data=df)
fitted2 = m2.fit(cov_type='HC1')

ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans


In [180]:
sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1]

1.1197458896534061e-228

In [181]:
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

F=0.919236, p=0.467231, k1=5.000000


In [185]:
str_regression = 'ceb ~ '
first = True
for column in df.columns:
    if column in ['religion', 'radio', 'tv']:
        continue
    if column in ['usemeth_noans', 'usemeth']:
        continue
    if column != 'ceb':
        plus = ' + '
        if first:
            plus = ''
        str_regression = str_regression + plus + column
        first = False

m3 = smf.ols(str_regression, data=df)
fitted3 = m3.fit(cov_type='HC1')

In [186]:
m2.fit().compare_f_test(m3.fit())

(92.890582301097126, 3.155200948040263e-40, 2.0)

In [187]:
fitted3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Thu, 10 Aug 2017   Prob (F-statistic):               0.00
Time:                        18:58:31   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.4612      0.350     -4.177      0.000      -2.147      -0.775
age                0.1776      0.004     41.592      0.000       0.169       0.186
educ              -0.0560      0.007     -7.788      0.000      -0.070      -0.042
idlnchld           0.0705      0.015      4.748      0.000       0.041       0.100
knowmeth           0.8739      0.174      5.029      0.000       0.533       1.214
agefm             -0.0649      0.010     -6.489      0.000      -0.085      -0.045
heduc             -0.0521      0.009     -5.658      0.000      -0.070      -0.034
urban             -0.1866      0.046     -4.019      0.000      -0.278      -0.096
electric          -0.3218      0.065     -4.953      0.000      -0.449      -0.194
bicycle            0.1979      0.048      4.083      0.000       0.103       0.293
nevermarr         -2.3625      0.205    -11.498      0.000      -2.765      -1.960
idlnchld_noans    -0.5266      0.225     -2.343      0.019      -0.967      -0.086
heduc_noans        0.7947      0.195      4.070      0.000       0.412       1.177
==============================================================================
Omnibus:                      250.641   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              936.515
Skew:                          -0.158   Prob(JB):                    4.35e-204
Kurtosis:                       5.251   Cond. No.                         428.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""